# Purpose of this notebook

This notebook covers how to start the Neuroglancer viewer, load in some data, and customize the Neuroglancer window all in Python. It assumes you already have a local copy of the Neuroglancer layer directories for the raw data, raw atlas and raw cells. These are on bucket here:


- raw data layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawdata_an21/
- raw atlas layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawatlas_an21/
- raw cells layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawcells_an21_dilated/

All together than are about 8 GB so copy them to your local machine if possible. It will speed up the load into Neuroglancer enormously because you will not need to use the VPN. 

## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like the raw data layer above) or type "segmentation" (like an atlas layer or cell layer). The naming is a little confusing because both layer types refer to volumes (3-d arrays). In Neuroglancer, you can overlay multiple layers or view them side-by-side, turn them on and off, and lots more. See the tutorial at [https://brainmaps.princeton.edu/2020/05/getting-started-with-neuroglancer/](https://brainmaps.princeton.edu/2020/05/getting-started-with-neuroglancer/) to learn more about what you can do with Neuroglancer.

# Setup
In order to run the code in this notebook, you will need an anaconda environment with python3 and containing some additional libraries. This environment "ng_ymaze" can be set up in the following way:
In terminal:
- conda create -n ng_ymaze python=3.7.4 -y
- conda activate ng_ymaze # (or source activate ng_mriatlas, depending on which version of conda you have)
- git clone https://github.com/austinhoag/cloud-volume # this fork has the fix for windows paths while we wait for the official seung-lab cloudvolume repo to incorporate the changes 
- cd cloud-volume
- pip install -e . 
- **pip install neuroglancer==1.1.6** <br>
- pip install --user ipykernel # to enable using this conda environment as a jupyter kernel
- python -m ipykernel install --user --name=ng_ymaze

At this point, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the notebook server, i.e. re-run jupyter notebook from the terminal)

## Host your layers on your local computer's network
This is a lot easier than it sounds. Open up a new terminal window on your computer and enter your ng_ymaze conda environment:
- conda activate ng_ymaze

Now enter a python interactive shell:
- python

Now execute the following lines in the python session:
```python
###WINDOW 1###
#in a new ipython window:
brainname = "20200930_17_32_58_hsv_36hr_7" #"20150804_tp_bl6_ts04"
port=1352
from cloudvolume import CloudVolume
import os
layer_dir = "/jukebox/scratch/zmd/save/contra_ipsi_projection_studies_20191125/%s/642" % brainname
#src = "/jukebox/LightSheetData/lightserv/jverpeut/natneuroreviews_tompisano_HSV-H129/natneuroreviews_tompisano_HSV-H129_20hr-006/imaging_request_1/viz/"
#layer_dir = os.path.join(src, brainname, "647")
vol = CloudVolume(f"file://{layer_dir}")
vol.viewer(port=port)
```

This should cause your session to hang -- that is expected. 

Now open up another terminal window and do the same thing but for the raw atlas, e.g.:
- conda activate ng_ymaze
- python 

Then, in python:
```python
###WINDOW 2###
#to add another layer (aka the atlas), in a new ipython window:
brainname = "20200930_17_32_58_hsv_36hr_7" #"20150804_tp_bl6_ts04"
port=1352
from cloudvolume import CloudVolume
import os
layer_dir = "/jukebox/scratch/zmd/save/contra_ipsi_projection_studies_20191125/%s/atlas" % brainname
#src = "/jukebox/LightSheetData/lightserv/jverpeut/natneuroreviews_tompisano_HSV-H129/natneuroreviews_tompisano_HSV-H129_20hr-006/imaging_request_1/viz/"
#layer_dir = os.path.join(src, brainname, "atlas")
vol = CloudVolume(f"file://{layer_dir}")
vol.viewer(port=port+1) #make sure this port is different from the first    
```

## Load your layers into Neuroglancer and view them

In [13]:
import neuroglancer 
neuroglancer.set_static_content_source(url="https://nglancer.pni.princeton.edu")
brainname="PRV_28hr-011"
port=1354
viewer = neuroglancer.Viewer()

In [14]:
with viewer.txn() as s:
    s.layers["%s" % brainname] = neuroglancer.ImageLayer(source="precomputed://http://localhost:%s" % port
    )
print(viewer)
#this should add the above volume to the neuroglancer window

http://127.0.0.1:41067/v/d64e8c8176bbaf60c0670211ede0377653d25f69/


In [15]:
with viewer.txn() as s:
    s.layers["%s_atlas" % brainname] = neuroglancer.SegmentationLayer(source="precomputed://http://localhost:%s" % int(port+1)
    )
print(viewer)
#this should add the atlas volume to the neuroglancer window

http://127.0.0.1:41067/v/d64e8c8176bbaf60c0670211ede0377653d25f69/


In [19]:
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
    s.show_panel_borders = False

AttributeError: 'str' object has no attribute 'config_state'

In [5]:
with viewer.config_state.txn() as s: 
    s.show_ui_controls = True 
    s.show_panel_borders = True 

In [ ]:
# old screenshot way
ss = neuroglancer.ScreenshotSaver(viewer, '/home/wanglab/Desktop')
ss.capture() 

In [ ]:
import os
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
    s.show_panel_borders = False
svdst = "/home/wanglab/Desktop/%s/prv_coronal_wo_overlay "% brainname
#make sure these directories exist
if not os.path.exists(os.path.dirname(svdst)): os.mkdir(os.path.dirname(svdst)) #brain directory
if not os.path.exists(svdst): os.mkdir(svdst) #structure directory
for i in range(4000,5000,50):
    if i%10==0: print(i)
    with viewer.txn() as s:
        s.voxel_coordinates = [2927,i,723] #the xy coords here are from the neuroglancer window
    s = neuroglancer.ScreenshotSaver(viewer, os.path.join(svdst, "%06d.png" % i))
    ss.capture()
    #ebdriver.driver.save_screenshot(os.path.join(svdst, "%06d.png" % i))

In [ ]:
# Start the webdriver which should open a new window
import os, time
from neuroglancer import webdriver
webdriver = webdriver.Webdriver(viewer, headless=False,)

In [ ]:
svdst = "/home/wanglab/Desktop/%s/prv_coronal_wo_overlay "% brainname
#make sure these directories exist
if not os.path.exists(os.path.dirname(svdst)): os.mkdir(os.path.dirname(svdst)) #brain directory
if not os.path.exists(svdst): os.mkdir(svdst) #structure directory

In [ ]:
for i in range(2000,3000,50):
    with viewer.txn() as s:
        s.voxel_coordinates = [2927,i,723] #the xy coords here are from the neuroglancer window
    time.sleep(30)
    webdriver.driver.save_screenshot(os.path.join(svdst, "%06d.png" % i))
    #s = neuroglancer.ScreenshotSaver(viewer, svdst)
    #ss.capture(i)
    print(i)

In [ ]:
# Turn off atlas layer
with webdriver.viewer.txn() as s:
    annotlayer = s.layers["%s_atlas" % brainname]
    annotlayer.visible=False

In [ ]:
#get more screenshots
svdst = "/home/wanglab/Desktop/%s/dcn_wo_overlay "% brainname
#make sure these directories exist
if not os.path.exists(os.path.dirname(svdst)): os.mkdir(os.path.dirname(svdst)) #brain directory
if not os.path.exists(svdst): os.mkdir(svdst) #structure directory

for i in range(5166,6056,20):
    if i%10==0: print(i)
    with viewer.txn() as s:
        s.voxel_coordinates = [2788,i,749] #the xy coords here are from the neuroglancer window
        #(where the L center scale is located)
    #optionally limit window size
#    with viewer.config_state.txn() as s:
#        s.viewer_size = [1000,1000]
    time.sleep(40)
    webdriver.driver.save_screenshot(os.path.join(svdst, "%06d.png" % i))

In [ ]:
#after, return controls to neuroglancer browser
with viewer.config_state.txn() as s: 
    s.show_ui_controls = True 
    s.show_panel_borders = True 